##  Combining Pandas and GeoPandas

In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np

### 1.- Loading world cities dataset

In [2]:
url = "https://github.com/opengeos/datasets/releases/download/world/world_cities.csv"
worldCities = pd.read_csv(url)
print('Number of cities',worldCities['id'].count())
worldCities.head()

Number of cities 1249


,id,name,country,latitude,longitude,population
0,1,Bombo,UGA,0.5833,32.5333,75000
1,2,Fort Portal,UGA,0.6710,30.2750,42670
2,3,Potenza,ITA,40.6420,15.7990,69060
3,4,Campobasso,ITA,41.5630,14.6560,50762
4,5,Aosta,ITA,45.7370,7.3150,34062


### 2.- including only cities with latitude values between -10 and 60 (i.e., cities located in the Northern Hemisphere or near the equator).


In [3]:
northenHemisphereCities = worldCities[(worldCities['latitude'] > -10.0) & (worldCities['latitude'] < 60.0)]
print('Number of cities in the northen hemisphere',northenHemisphereCities['id'].count())
northenHemisphereCities.head()

Number of cities in the northen hemisphere 956


,id,name,country,latitude,longitude,population
0,1,Bombo,UGA,0.5833,32.5333,75000
1,2,Fort Portal,UGA,0.6710,30.2750,42670
2,3,Potenza,ITA,40.6420,15.7990,69060
3,4,Campobasso,ITA,41.5630,14.6560,50762
4,5,Aosta,ITA,45.7370,7.3150,34062


### 3.-  Creating a GeoDataFrame from the filtered dataset by converting the latitude and longitude into geometries.

In [4]:
northenHemisphereCitiesGDP = gpd.GeoDataFrame(
    northenHemisphereCities,geometry=gpd.points_from_xy(northenHemisphereCities.longitude, northenHemisphereCities.latitude)
)
northenHemisphereCitiesGDP.crs="EPSG:4326"
northenHemisphereCitiesGDP.head()

,id,name,country,latitude,longitude,population,geometry
0,1,Bombo,UGA,0.5833,32.5333,75000,POINT (32.53330 0.58330)
1,2,Fort Portal,UGA,0.6710,30.2750,42670,POINT (30.27500 0.67100)
2,3,Potenza,ITA,40.6420,15.7990,69060,POINT (15.79900 40.64200)
3,4,Campobasso,ITA,41.5630,14.6560,50762,POINT (14.65600 41.56300)
4,5,Aosta,ITA,45.7370,7.3150,34062,POINT (7.31500 45.73700)


### 4.- Reprojecting the GeoDataFrame to the Mercator projection (EPSG:3857).


In [5]:
northenHemisphereCitiesGDP_3857 = northenHemisphereCitiesGDP.to_crs("EPSG:3857")
northenHemisphereCitiesGDP_3857.head()

,id,name,country,latitude,longitude,population,geometry
0,1,Bombo,UGA,0.5833,32.5333,75000,POINT (3621590.390 64933.781)
1,2,Fort Portal,UGA,0.6710,30.2750,42670,POINT (3370197.584 74697.086)
2,3,Potenza,ITA,40.6420,15.7990,69060,POINT (1758736.635 4959679.293)
3,4,Campobasso,ITA,41.5630,14.6560,50762,POINT (1631498.457 5095742.084)
4,5,Aosta,ITA,45.7370,7.3150,34062,POINT (814302.075 5738302.989)


### 5.- Calculating the distance (in meters) between each city and the city of Paris.


In [6]:
# Set name as the index for easier reference
northenHemisphereCitiesGDP_3857 = northenHemisphereCitiesGDP_3857.set_index("name")
paris_coordinates = northenHemisphereCitiesGDP_3857.loc['Paris','geometry']
print(paris_coordinates)

POINT (259746.220647577 6252271.673730295)


In [7]:
northenHemisphereCitiesGDP_3857["distance_to_paris"] = northenHemisphereCitiesGDP_3857["geometry"].distance(paris_coordinates)


In [8]:
northenHemisphereCitiesGDP_3857.head()

,id,country,latitude,longitude,population,geometry,distance_to_paris
name,,,,,,,
Bombo,1,UGA,0.5833,32.5333,75000,POINT (3621590.390 64933.781),7.041672e+06
Fort Portal,2,UGA,0.6710,30.2750,42670,POINT (3370197.584 74697.086),6.916454e+06
Potenza,3,ITA,40.6420,15.7990,69060,POINT (1758736.635 4959679.293),1.979335e+06
Campobasso,4,ITA,41.5630,14.6560,50762,POINT (1631498.457 5095742.084),1.794231e+06
Aosta,5,ITA,45.7370,7.3150,34062,POINT (814302.075 5738302.989),7.561058e+05


In [9]:
northenHemisphereCitiesGDP_3857 = northenHemisphereCitiesGDP_3857.set_index("id")
northenHemisphereCitiesGDP_3857.head()

,country,latitude,longitude,population,geometry,distance_to_paris
id,,,,,,
1,UGA,0.5833,32.5333,75000,POINT (3621590.390 64933.781),7.041672e+06
2,UGA,0.6710,30.2750,42670,POINT (3370197.584 74697.086),6.916454e+06
3,ITA,40.6420,15.7990,69060,POINT (1758736.635 4959679.293),1.979335e+06
4,ITA,41.5630,14.6560,50762,POINT (1631498.457 5095742.084),1.794231e+06
5,ITA,45.7370,7.3150,34062,POINT (814302.075 5738302.989),7.561058e+05


In [10]:
northenHemisphereCitiesGDP["distance_to_paris"] = northenHemisphereCitiesGDP_3857["distance_to_paris"]
northenHemisphereCitiesGDP.head()

,id,name,country,latitude,longitude,population,geometry,distance_to_paris
0,1,Bombo,UGA,0.5833,32.5333,75000,POINT (32.53330 0.58330),NaN
1,2,Fort Portal,UGA,0.6710,30.2750,42670,POINT (30.27500 0.67100),7.041672e+06
2,3,Potenza,ITA,40.6420,15.7990,69060,POINT (15.79900 40.64200),6.916454e+06
3,4,Campobasso,ITA,41.5630,14.6560,50762,POINT (14.65600 41.56300),1.979335e+06
4,5,Aosta,ITA,45.7370,7.3150,34062,POINT (7.31500 45.73700),1.794231e+06


### 6.- Plotting the cities on a world map, coloring the points by their distance from Paris.


In [11]:
northenHemisphereCitiesGDP.explore('distance_to_paris')